In [1]:
# Import necessary modules
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import plotly.express as px 


# Import CRUD module (ensure correct module name)
from aac_crud import AnimalShelterCRUD

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Serendipity"
HOST = 'nv-desktop-services.apporto.com'
PORT = 33676
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelterCRUD(username, password, HOST, PORT, DB, COL)

# Fetch initial dataset (only dogs)
df = pd.DataFrame.from_records(db.read({'animal_type': 'Dog'})  # Fetch data first
)
df = df.sort_values(by="rec_num", ascending=True)  # Sort explicitly

# Handle MongoDB ObjectID issue
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Set default row selection to the first available row (if dataset exists)
selected_row_default = [0] if not df.empty else []

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load logo image
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(
        className='row',
        style={
            'display': 'flex',
            'justify-content': 'center',
            'align-items': 'center',
            'margin': '9px',
            'width': '100%'
        },
        children=[
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height': '10%', 'width': '10%', 'margin-right': '15px'}
            ),
            html.Div(
                html.B(html.H1('Grazioso Salvare Dashboard - CS340 - Ivette Cerpa')),
                style={'text-align': 'center'}
            )
        ]
    ),
    
    html.Hr(),

    #########################
    # Interactive Filtering
    #########################
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': '  Water Rescue  ', 'value': 'Water'},
            {'label': '  Mountain and Wilderness Rescue  ', 'value': 'Mountain'},
            {'label': '  Disaster and Individual Rescue  ', 'value': 'Disaster'},
            {'label': '  Reset  ', 'value': 'Reset'}
        ],
        value='Reset',
        labelStyle={'display': 'inline-block', 'padding': '10px'}
    ),
    
    html.Hr(),

    #########################
    # Data Table
    #########################
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=selected_row_default,  # Dynamically selects first row if data exists
        page_action="native",
        page_current=0,
        page_size=10
    ),
    
    html.Br(),
    html.Hr(),

    #########################
    # Visualization Layout
    #########################
    html.Div(className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

#########################
# Data Filtering Callback
#########################
@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'selected_rows')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type is None:
        return [], []

    # Initialize selected_row_default safely
    selected_row_default = None

    df_filtered = None

    if filter_type == 'Water':
        df_filtered = pd.DataFrame.from_records(db.read(
            {"$and": [
                {"$or": [
                    {'breed': {"$regex": 'Labrador Retriever Mix'}},
                    {'breed': {"$regex": 'Chesapeake'}},
                    {'breed': {"$regex": 'Newfoundland'}}
                ]},
                {'sex_upon_outcome': 'Intact Female'},
                {"$and": [
                    {'age_upon_outcome_in_weeks': {"$gt": 26}},
                    {'age_upon_outcome_in_weeks': {"$lt": 156}}
                ]}
            ]}
        ))
    elif filter_type == 'Mountain':
        df_filtered = pd.DataFrame.from_records(db.read(
            {"$and": [
                {"$or": [
                    {'breed': {"$regex": 'German Shepherd'}},
                    {'breed': {"$regex": 'Alaskan Malamute'}},
                    {'breed': {"$regex": 'Old English Sheepdog'}},
                    {'breed': {"$regex": 'Siberian Husky'}},
                    {'breed': {"$regex": 'Rottweiler'}}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {"$and": [
                    {'age_upon_outcome_in_weeks': {"$gt": 26}},
                    {'age_upon_outcome_in_weeks': {"$lt": 156}}
                ]}
            ]}
        ))
    elif filter_type == 'Disaster':
        df_filtered = pd.DataFrame.from_records(db.read(
            {"$and": [
                {"$or": [
                    {'breed': {"$regex": 'Doberman Pinscher'}},
                    {'breed': {"$regex": 'German Shepherd'}},
                    {'breed': {"$regex": 'Golden Retriever'}},
                    {'breed': {"$regex": 'Bloodhound'}},
                    {'breed': {"$regex": 'Rottweiler'}}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {"$and": [
                    {'age_upon_outcome_in_weeks': {"$gt": 20}},
                    {'age_upon_outcome_in_weeks': {"$lt": 300}}
                ]}
            ]}
        ))
    elif filter_type == 'Reset':  # **Explicitly handle Reset here**
        df_filtered = pd.DataFrame.from_records(db.read({'animal_type': 'Dog'}))

    # Apply sorting after filtering
    if df_filtered is not None and '_id' in df_filtered.columns:
        df_filtered['_id'] = df_filtered['_id'].astype(str)  # Convert ObjectId to string
        df_filtered.drop(columns=['_id'], inplace=True)  # Drop if unnecessary
        df_filtered = df_filtered.sort_values(by="rec_num", ascending=True)  # **Sort here**

    # Ensure valid output format
    data_output = df_filtered.to_dict('records') if not df_filtered.empty else []

    # Fix selected_row_default handling for all cases
    selected_row_default = [0] if not df_filtered.empty else []

    return data_output, selected_row_default

#########################
# Pie Chart Callback
#########################
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData or len(viewData) == 0:
        return []

    df = pd.DataFrame.from_records(viewData)

    # Limit to the top 10 most frequent breeds
    if 'breed' in df.columns:
        top_breeds = df['breed'].value_counts().nlargest(10).index.tolist()
        df = df[df['breed'].isin(top_breeds)]

    # Ensure correct display size
    fig = px.pie(df, names='breed', title='Preferred Animals')
    fig.update_layout(height=600, width=600)

    return [
        dcc.Graph(figure=fig)
    ]



#########################
# Map Callback
#########################
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if not viewData or len(viewData) == 0:
        return []

    df = pd.DataFrame.from_records(viewData)
    row = index[0] if index else 0

    if row >= len(df):
        row = 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[df.iloc[row, 13], df.iloc[row, 14]],
                    children=[
                        dl.Tooltip(df.iloc[row, 4]),
                        dl.Popup([html.H1("Animal Name"), html.P(df.iloc[row, 9])])
                    ])
            ])
    ]

#########################
# Run Server
#########################
app.run_server(debug=True)


Dash app running on http://127.0.0.1:13181/
